In [1]:
%pip install pynytimes
#!/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Cpntext of article (Time and place and people)
entity recognition, to pull locations in the article

In [2]:

from pynytimes import NYTAPI
import pandas as pd
from datetime import date

api_key = "" #insert API key
nyt=NYTAPI(key=api_key, parse_dates=True)

Creating lists for variables that are relevant and going through 200 articles, and for any article that is about gentrification, getting all the data and creating a dataframe

In [11]:
#set up empty sets for each field
abstracts=[]
weburls=[]
snippets=[]
lead_paragraphs=[]
headlines=[]
pubdates=[]
wordcounts=[]
place_keywords=[]
people_keywords=[]
subject_keywords=[]
gentrification_keyword=[]
gentrification_headline=[]

#get 2000 articles
#change this based on start and end date
articles = nyt.article_search(query = "gentrification",results=2000)

/usr4/ugrad/manavk/.local/lib/python3.8/site-packages/pynytimes/helpers/article_search.py:16: UserWarning: Asking for a lot of results, because of rate limits it can take a while.
  warnings.warn(


In [13]:
#adding the fields for each article to the respective lists
for article in articles:
    abstracts.append(article['abstract'])
    weburls.append(article['web_url'])
    snippets.append(article['snippet'])
    lead_paragraphs.append(article['lead_paragraph'])
    headlines.append(article['headline']['main'])
    pubdates.append(article['pub_date'])
    wordcounts.append(article['word_count'])


    #sorting the keywords and adding to the data frame
    keywords=article['keywords']
    s=[]
    pl=[]
    pe=[]
    for i in keywords:
      if i['name']=='subject':
        s.append(i['value'])
      elif i['name']=='persons':
        pe.append(i['value'])
      elif i['name']=='glocations':
        pl.append(i['value'])
    if pl==[]:
      place_keywords.append('NA')
    else:
      place_keywords.append(pl)
    if pe==[]:
      people_keywords.append('NA')
    else:
      people_keywords.append(pe)
    if s==[]:
      subject_keywords.append('NA')
    else:
      subject_keywords.append(s)
    
    
#extra field for chcking if gentrification is in the healdine/is a keyword 
    if 'Gentrification' not in s:
      gentrification_keyword.append(False)
    else:
      gentrification_keyword.append(True)

    
    if "gentrification" in article["lead_paragraph"] or "gentrification" in article["headline"]:
      gentrification_headline.append(True)
    else:
      gentrification_headline.append(False)

    
    

In [14]:
#create a dataframe with the required fields
data = {'Headlines': headlines,
        'Lead paragraph': lead_paragraphs,
        'abstract': abstracts,
        #'Snippet': snippets,
        #'Web Url': weburls,
        #"Publishing date": pubdates,
        #'Word count': wordcounts,
        'places':place_keywords,
        'people':people_keywords,
        'topics':subject_keywords,
        'Is gentrification a keyword':gentrification_keyword,
        'gentrification in headline or lead para':gentrification_headline}

df=pd.DataFrame(data)
  

entity recognition:
  spacee package
  https://huggingface.co/spacy/en_core_web_lg
  https://huggingface.co/dslim/bert-base-NER


In [15]:
df

,Headlines,Lead paragraph,abstract,places,people,topics,Is gentrification a keyword,gentrification in headline or lead para
0,"As Austin’s SoCo District Grows, Is It Losing ...","AUSTIN, Texas — In the early half of the 20th ...",Streets like South Congress Avenue are crucial...,[Austin (Tex)],NA,"[Shopping and Retail, Gentrification, Area Pla...",True,False
1,Why Harlem Is Getting a Truck Depot Instead of...,The lot on West 145th Street in Manhattan was ...,A developer wanted to build residential towers...,"[Harlem (Manhattan, NY)]","[Teitelbaum, Bruce J, Jordan, Kristin Richards...","[Trucks and Trucking, Real Estate and Housing ...",True,False
2,Must We Gentrify the Rest Stop?,"Five years ago, the New York State Thruway Aut...","McDonald’s is gone, and the Manhattanization o...","[Dewey, Thomas E, Thruway, New York State]",NA,"[Roads and Traffic, Gentrification, Fast Food ...",True,False
3,Remaking the River That Remade L.A.,"Over the past century it has been channeled, s...","Over the past century it has been channeled, s...","[Los Angeles River (Calif), Los Angeles (Calif...",NA,"[Area Planning and Renewal, Gentrification, Fl...",True,False
4,"A Legendary World Builder on Multiverses, Revo...",N.K. Jemisin is a fantasy and science-fiction ...,How the fantasy and science-fiction writer N.K...,NA,"[Jemisin, N K]","[audio-neutral-informative, Writing and Writer...",True,False
...,...,...,...,...,...,...,...,...
1995,"Review: ‘The Last O.G.’ Searches for Brooklyn,...",“The Last O.G.” is a time-travel comedy. It ju...,Tracy Morgan and Tiffany Haddish are well-pair...,NA,"[Morgan, Tracy, Haddish, Tiffany, Peele, Jordan]",[Television],False,False
1996,"Little Italy Is Very Little, and Not Very Italian","On Grand Street, the Banh Mi Saigon Vietnamese...",A fire at the oldest Italian restaurant in the...,"[Little Italy (Manhattan, NY)]",NA,"[Italian-Americans, Renting and Leasing (Real ...",False,False
1997,NEIGHBORHOOD REPORT: LOWER MANHATTAN;Rabbi See...,Gone are the days when there was a synagogue o...,Gone are the days when there was a synagogue o...,"[New York City, Lower East Side (NYC)]","[Spiegel, Jacob]","[JEWS, Historic Buildings and Sites, RESTORATI...",False,False
1998,The National Crackup,"I never cared for the “melting pot” metaphor, ...","We are retreating to our tribal, ethnic and pr...",NA,"[Trump, Donald J, Obama, Barack]","[Race and Ethnicity, Discrimination, United St...",False,False


In [17]:
df.to_csv('NYT_data.csv')